# School Budget Model Data

6/8/2019

In [1]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/fc/finance_subcommittee/google/budg


In [2]:
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Set max_rows option for display

In [3]:
pd.get_option("display.max_rows")

60

In [4]:
pd.set_option("display.max_rows",1000)
pd.get_option("display.max_rows")

1000

## The following code to reads the RI Uniform Charter of Accounts data from the Rhode Island Department of Education Consolidated District Financial Database

See:

http://www.ride.ri.gov/FundingFinance/SchoolDistrictFinancialData/UniformChartofAccounts.aspx#18211078-statewide-consolidated-district-financial-databases

### Read the FY2017  (2016-2017 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY17.xlsx

In [5]:
fy17 = pd.read_csv("../../../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY17.csv")
fy17['Year'] = 2017
print(fy17.shape)
print(fy17.columns)
fy17eg = fy17.loc[fy17['Dist No'] == 90.0]
print(fy17eg.shape)

(215694, 23)
Index(['ID', 'Dist No', 'District Name', 'Fund', 'Fund Description',
       'Loc Type', 'Loc Type desc', 'Loc', 'Location Description', 'Func',
       'Function Description', 'Prog', 'Program Description', 'Sub',
       'Subject Description', 'Obj', 'Object Description', 'JC',
       'Job Class Description', 'Actual', 'Budget', 'School Enrollment',
       'Year'],
      dtype='object')
(2951, 23)


In [6]:
fy17eg['LocD'] = fy17eg['Loc'].astype('str') + ' ' + fy17eg['Location Description']
fy17eg['FuncD'] = fy17eg['Func'].astype('str') + ' ' + fy17eg['Function Description']
fy17eg['SubD'] = fy17eg['Sub'].astype('str') + ' ' + fy17eg['Subject Description']
fy17eg['ProgD'] = fy17eg['Prog'].astype('str') + ' ' + fy17eg['Program Description']
fy17eg['JCD'] = fy17eg['JC'].astype('str') + ' ' + fy17eg['Job Class Description']
fy17eg['FundD'] = fy17eg['Fund'].astype('str') + ' ' + fy17eg['Fund Description']
fy17eg['ObjD'] = fy17eg['Obj'].astype('str') + ' ' + fy17eg['Object Description']
fy17eg2 = fy17eg.drop(['ID','District Name','Loc','Loc Type','Location Description','Func','Function Description', \
                     'Loc Type desc','Fund','Fund Description','Sub','Subject Description', \
                     'Job Class Description','Prog','Program Description','Object Description', \
                     'Dist No','School Enrollment'],axis=1)
fy17eg2

/home/gquinn/.conda/envs/google/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gquinn/.conda/envs/google/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gquinn/.conda/envs/google/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,Obj,JC,Actual,Budget,Year,LocD,FuncD,SubD,ProgD,JCD,FundD,ObjD
48979,52501,1100,461.01,652.92,2017,5106 East Greenwich High School,111 Instructional Teachers,1700 Natural Sciences,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48980,52501,1100,402.12,576.13,2017,5106 East Greenwich High School,111 Instructional Teachers,500 English Language Arts,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48981,52501,1100,404.18,560.00,2017,5106 East Greenwich High School,111 Instructional Teachers,1500 Mathematics,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48982,52501,1100,388.28,540.91,2017,5106 East Greenwich High School,111 Instructional Teachers,1900 Social Sciences,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48983,52501,1100,372.35,529.67,2017,4103 Archie R. Cole Middle School,111 Instructional Teachers,1700 Natural Sciences,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48984,52501,1100,372.45,520.98,2017,4103 Archie R. Cole Middle School,111 Instructional Teachers,1900 Social Sciences,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48985,52501,1100,362.57,497.72,2017,4103 Archie R. Cole Middle School,111 Instructional Teachers,1500 Mathematics,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48986,52501,1100,353.69,487.51,2017,5106 East Greenwich High School,111 Instructional Teachers,700 Foreign Languages,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48987,52501,1100,337.76,476.04,2017,4103 Archie R. Cole Middle School,111 Instructional Teachers,500 English Language Arts,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance
48988,52501,1100,327.23,484.72,2017,4103 Archie R. Cole Middle School,111 Instructional Teachers,2101 General Education with Supports and Servi...,20 Special Education,1100 Teachers,10000000 General Fund,52501 Unemployment Insurance


In [ ]:
fy17egrs = fy17eg2.loc[(fy17eg2['Obj']==51110)]
fy17egrs

In [9]:
fy17egrs = fy17eg2.loc[(fy17eg2['Obj']==51110) & (fy17eg2['JC'].isin([1100,1300,1500,1600,1700]))]
fy17egrs

,Obj,JC,Actual,Budget,Year,LocD,FuncD,SubD,ProgD,JCD,FundD,ObjD
49322,51110,1100,0.00,20435.75,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,0 General Education,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49328,51110,1100,385757.51,391843.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,5 Grade 3,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49336,51110,1100,317735.02,326599.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,6 Grade 4,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49343,51110,1100,272703.00,300972.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,7 Grade 5,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49350,51110,1100,40205.10,40871.50,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,200 Art,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49357,51110,1100,117112.92,98257.40,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,1200 Physical Education and Health,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49364,51110,1100,67058.82,85830.15,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,1600 Music/Theatre and Performing Arts/Drama,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49372,51110,1100,81753.11,82866.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,2400 Literacy and Reading,10 Regular Elementary/Secondary Education Prog...,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49379,51110,1100,161209.62,156066.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,2101 General Education with Supports and Servi...,20 Special Education,1100 Teachers,10000000 General Fund,51110 Regular Salaries
49386,51110,1100,79590.90,82438.00,2017,3102 James H. Eldredge El. School,111 Instructional Teachers,2103 Special Class - School Year (Census Cost ...,20 Special Education,1100 Teachers,10000000 General Fund,51110 Regular Salaries


### Read the FY2016  (2015-2016 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY16.xlsx

In [ ]:
fy16 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY16.csv")
fy16['Year'] = 2016
print(fy16.shape)
fy16.columns

### Read the FY2015  (2014-2015 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY15.xlsx

In [ ]:
fy15 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY15.csv")
fy15['Year'] = 2015
print(fy15.shape)
fy15.columns

### Read the FY2014  (2013-2014 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-081415.xlsx

In [ ]:
fy14 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-081415.csv")
fy14['Year'] = 2014
print(fy14.shape)
fy14.columns

### Read the FY2013  (2012-2013 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY13.xlsx

In [ ]:
fy13 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY13.csv")
fy13['Year'] = 2013
print(fy13.shape)
fy13.columns

### Read the FY2012  (2011-2012 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422.xlsx

In [ ]:
fy12 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422.csv")
fy12['Year'] = 2012
print(fy12.shape)
fy12.columns

### Read the FY2011  (2010-2011 school year) data: Expenditures Excluding Capital and Debt Service 

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY11.xlsx

In [ ]:
fy11 = pd.read_csv("../../ride/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY11.csv")
fy11['Year'] = 2011
print(fy11.shape)
fy11.columns

### Read the FY2010  (2009-2010 school year) data: Expenditures Excluding Capital and Debt Service 

Note:  No Budget information in this year

Source file is: 

http://www.ride.ri.gov/Portals/0/Uploads/Documents/UCOA/2009-2010-Consolidated-UCOA-Database/FY-10-UCOA-Database-Expenditures-Excluding-Capital-and-Debt-Service.xlsx

In [ ]:
fy10 = pd.read_csv("../../ride/FY-10-UCOA-Database-Expenditures-Excluding-Capital-and-Debt-Service.csv")
fy10['Year'] = 2010
print(fy10.shape)
fy10.columns

In [ ]:
allexp = pd.concat([fy17,fy16,fy15,fy14,fy13,fy12,fy11,fy10],sort=False)
allexp['']
print(allexp.shape)

In [ ]:
allexp.to_csv("../../allexp.csv")

In [ ]:
alleg = allexp.loc[allexp['Dist No']==90.0]
alleg['delta'] = alleg['Budget'] > alleg['Actual']
alleg.to_csv("../../alleg.csv")

In [ ]:
alleg = pd.read_csv("../../alleg.csv")
print(alleg.shape)
alleg.columns

In [ ]:
alleg.loc[(alleg['Obj']==51110.0)&(alleg['Func']==212.0)&(alleg['Loc']==4103.0)][['Year',  \
                        'Loc','Function Description','Subject Description','Budget','Actual','delta']]